In [2]:
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException

from difflib import SequenceMatcher
from bs4 import BeautifulSoup
############################################################
import os, csv
import time


class search_save_CSV:
    def __init__(self, i=0):
        
        header = [\
            "user-agent=Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3", \
            "user-agent=Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36", \
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome", \
        ]
        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument(header[i])
        options.add_argument('Accept-Charset=utf-8')
        
        self.driver = webdriver.Chrome("./chromedriver.exe", options=options)
        self.driver.implicitly_wait(5)
        
    def getKeyword_andOpen(self, search_text):
        self.driver.get("https://m.map.naver.com/")
        
        self.search_ = search_text
        self.driver.find_element_by_class_name("Nsearch._searchKeywordView._searchGuide").click()
        
        self.driver.find_element_by_class_name("Nbox_input_text._search_input").send_keys(self.search_)
        self.driver.find_element_by_class_name("_formSearch").submit()
        
    def errorCheck(self):
        # print(self.driver.find_element_by_class_name('interstitial-wrapper'))
        try:
            self.driver.find_element_by_class_name('interstitial-wrapper')
            print("error-reload")
            return True
        except NoSuchElementException:
            return False
        
    def search_and_save(self):
        searchList = self.driver.find_elements_by_class_name("_item._lazyImgContainer")
        choicePer, choiceIndex = 0, -1
        for i, element in enumerate(searchList):
            # print(i, ":", element.get_attribute("data-title"))    
            ratio = self.getMatchPercent(element.get_attribute("data-title"), self.search_)
            print("{0:.1f}%".format(ratio))
            
            if ratio < 50:
                pass
            else:
                if choicePer < ratio:
                    choicePer = ratio
                    choiceIndex = i
        if choiceIndex == -1:
            print("일치결과 없음")
            return False
        else:
            print("선택 index {0} : {1} 과의 일치율 {2:.1f}".format(choiceIndex, self.search_, choicePer))
            self.index = choiceIndex
            return True
        
    def get_menu_list(self): 
        elements = self.driver.find_elements_by_class_name("sp_map.btn_price._linkMenu")
        try:
            elements[self.index].click()
        except Exception:
            "가격정보 없음"
            return False
        time.sleep(5)
        
        temp = str(self.driver.current_url)
        temp = temp[:-4] + 'menu/list'
        print(temp)
        self.driver.get(temp)
        
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        elements = soup.find_all("li", class_="_1jXsR")
        menuName = soup.find_all("span", class_="_1zr5s")
        menuPrice = soup.find_all("div", class_="_1cQ4a")

        for i, menu in enumerate(menuName):
            print(i, menu.text, ":", menuPrice[i].text)
            # menuPrice[i].text = menuPrice[i].text.replace(",", "")
        self.name = menuName
        self.price = menuPrice
        return True
        
    def save_to_csv(self, pk):
        saveFileName = os.path.abspath("../data/price.csv")
        if not os.path.exists(saveFileName):
            file = open(saveFileName, "w", newline="", encoding='utf-8')
            newWriter = csv.writer(file)
            newWriter.writerow(["관리번호", "메뉴이름", "가격"])
            file.close()
        file = open(saveFileName, "a", newline="", encoding='utf-8')
        writer = csv.writer(file)
        prices = [i.text for i in self.price]
        for i, menu in enumerate(self.name):
            row = [str(pk), menu.text, prices[i].replace(',', '')]
            writer.writerow(row)
        print("save success")
        file.close()      
        
        
    def getMatchPercent(self, one, two):
        s = SequenceMatcher(None, one, two)
        return s.ratio() * 100
    
    def close_driver(self):
        self.driver.quit()

In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("coffee-price").setMaster("local[*]")
sc = SparkContext(conf=conf).getOrCreate()

In [3]:
import os

data_path = os.path.join("..", "data", "data.csv")
raw_data = sc.textFile(data_path)
raw_data_header = raw_data.take(1)[0]

In [4]:
headerList = str(raw_data_header).split(",") # 관리번호 = PK
need_column = ['관리번호', '영업상태구분코드', '영업상태명', '소재지전화',\
        '소재지면적','소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점',\
        '업태구분명', '좌표정보(X)', '좌표정보(Y)', '시설총규모', '홈페이지']
need_column_index = []
column_list = []
for i, value in enumerate(headerList):
    if str(value) in need_column:
        need_column_index.append(i)
        column_list.append(str(value))

In [5]:
data = raw_data.filter(lambda line: line!=raw_data_header) \
    .map(lambda line: line.split(",")) \
    .filter(lambda tokens: tokens[24] == '커피숍' and tokens[7] =='영업/정상') \
    .map(lambda tokens: [tokens[i] for i in need_column_index]) \
    .filter(lambda tokens: tokens[5].split()[0] == '경기도' or tokens[5].split()[0] == '충청북도')

In [6]:
elements = data.take(100000)
print(len(elements))

16000


In [7]:
elements = data.take(100000)
dongList = list()
num = 0
search = search_save_CSV(num)

for i, element in enumerate(elements):
    if i < 12431:
        continue
    print(i, "번째.")
#     if i%10 == 9:
#         search.close_driver()
#         time.sleep(1)
#         num = num + 1
#         search = search_save_CSV(num % 3)
    for i in range(len(column_list)):
        print("[%s]" % i, column_list[i], ":", element[i])
    continue_list = \
    ["스타벅스", "이디야", "투썸", "투썸플레이스", "엔젤리너스", "커피빈", "구스토", "달콤커피", "메가커피", \
    "바나프레소", "빽다방", "셀렉토", "요거프레소", "주커피", "드롭탑", "띠아모", "카페베네", "봄봄", \
    "커핀그루나루", "커피마마", "커피에 반하다", "탐앤탐스", "폴 바셋", "할리스", "파스쿠찌", "쥬씨"]
    
    name_list = str(element[7]).split()
    flag = False
    for comp in name_list:
        for li in continue_list:
            if li in comp:
                flag = True
                break
        if flag:
            break
    if flag: # 프랜차이즈의 경우는 넘어간다.
        continue
    
    address = str(element[5]).split()
    try:
        dongList.append(address[2])
        print("************************************************")
        search_keyword = address[2] + " " + element[7]
        
        search.getKeyword_andOpen(search_keyword)
    
        
        # check error page
        while search.errorCheck():
            time.sleep(10)
            search.close_driver()
            num = num + 1
            search = search_save_CSV(num % 3)
            search.getKeyword_andOpen(search_keyword)
            
            
        flag_ = search.search_and_save()
        if flag_:
            if search.get_menu_list():
                time.sleep(1)
                search.save_to_csv(element[0])
    except Exception:
        continue
search.close_driver()

12431 번째.
[0] 관리번호 : 4390000-104-2017-00090
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 93.84
[5] 소재지전체주소 : 충청북도 충주시 대소원면 검단리 350-12번지
[6] 도로명전체주소 : 충청북도 충주시 대소원면 대학로 24-1  1층
[7] 사업장명 : 커피베이한국교통대점
[8] 최종수정시점 : 20170821114308
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.96484321564716
[11] 좌표정보(Y) : 127.8714153189772
[12] 시설총규모 : 93.84
[13] 홈페이지 : None
************************************************
일치결과 없음
12432 번째.
[0] 관리번호 : 4390000-104-2018-00086
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 23.14
[5] 소재지전체주소 : 충청북도 충주시 충의동 272번지
[6] 도로명전체주소 : 충청북도 충주시 중앙로 55 (충의동)
[7] 사업장명 : 더커피
[8] 최종수정시점 : 20180713143154
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.973100539084804
[11] 좌표정보(Y) : 127.92959423644186
[12] 시설총규모 : 23.14
[13] 홈페이지 : None
************************************************
42.9%
20.0%
일치결과 없음
12433 번째.
[0] 관리번호 : 4390000-104-2018-00087
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 257.21
[5] 소재지전체주소 : 충청북도 충주시 중앙탑면 용전리 670번지 골든스카이 타워

0 아메리카노 Hot / Ice : 3,000원
save success
12447 번째.
[0] 관리번호 : 4390000-104-2017-00138
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 312.08
[5] 소재지전체주소 : 충청북도 충주시 봉방동 76-1번지
[6] 도로명전체주소 : 충청북도 충주시 봉계13길 2  1-2층 (봉방동)
[7] 사업장명 : 스타벅스커피충주터미널점
[8] 최종수정시점 : 20190521170406
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.98059458855853
[11] 좌표정보(Y) : 127.91527307603658
[12] 시설총규모 : 312.08
[13] 홈페이지 : None
12448 번째.
[0] 관리번호 : 4390000-104-2017-00122
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 57.30
[5] 소재지전체주소 : 충청북도 충주시 엄정면 괴동리 138-1번지
[6] 도로명전체주소 : 충청북도 충주시 엄정면 밤밭1길 63
[7] 사업장명 : 카페밤밭2017
[8] 최종수정시점 : 20171023093439
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.11049527256709
[11] 좌표정보(Y) : 127.93326534749677
[12] 시설총규모 : 57.3
[13] 홈페이지 : None
************************************************
76.2%
선택 index 0 : 엄정면 카페밤밭2017 과의 일치율 76.2
https://m.place.naver.com/restaurant/1950620509/menu/list
0 2.수제 잠봉뵈르 바게트 세트 : 17,500원
1 1.바베큐그릴치킨햄버거세트 : 17,500원
2 3.베이컨 피타브레드 세트 : 18,500원
3 4. 냉

33.3%
일치결과 없음
12466 번째.
[0] 관리번호 : 4390000-104-2016-00117
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043854 3437
[4] 소재지면적 : 7.91
[5] 소재지전체주소 : 충청북도 충주시 대소원면 검단리 350-58번지
[6] 도로명전체주소 : 충청북도 충주시 대소원면 대학로 20  2층 3호
[7] 사업장명 : 쥬씨충주교통대점
[8] 최종수정시점 : 20200203135103
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.964527504007584
[11] 좌표정보(Y) : 127.87139553326492
[12] 시설총규모 : 7.91
[13] 홈페이지 : None
12467 번째.
[0] 관리번호 : 4390000-104-2016-00086
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 96.00
[5] 소재지전체주소 : 충청북도 충주시 교현동 464-2번지
[6] 도로명전체주소 : 충청북도 충주시 신호4길 33 (교현동  제니아건물 1층)
[7] 사업장명 : 프럼(FROM)105
[8] 최종수정시점 : 20161011144849
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.977087303204044
[11] 좌표정보(Y) : 127.94263006113682
[12] 시설총규모 : 96
[13] 홈페이지 : None
************************************************
60.9%
40.0%
선택 index 0 : 교현동 프럼(FROM)105 과의 일치율 60.9
https://m.place.naver.com/restaurant/84676403/menu/list
0 아메리카노 : 3,000원
1 화이트더치 : 5,000원
2 앙마식빵 : 2,500원
3 트리플치즈 : 2,000원
4 레즈벨벳 케이크 (1호) : 20,000원

81.5%
선택 index 0 : 칠금동 더벤티 롯데마트 충주점 과의 일치율 81.5
https://m.place.naver.com/restaurant/271434908/menu/list
0 [HOT]아메리카노 : 1,500원
1 [ICE]아메리카노 : 1,800원
2 믹스커피 : 1,800원
3 카푸치노 : 2,500원
4 카페라떼 : 2,500원
5 바닐라/헤즐넛라떼 : 2,800원
6 카라멜마끼야또 : 3,000원
7 카페모카 : 3,000원
8 콜드브루 : 3,000원
9 콜드브루 라떼 : 3,500원
10 연유라떼 : 3,500원
11 미숫가루 : 2,500원
12 복숭아 아이스티 : 3,000원
13 초코 / 녹차 / 홍차라떼 : 3,000원
14 딸기라떼 / 딸바라떼 : 3,500원
save success
12485 번째.
[0] 관리번호 : 4390000-104-2016-00077
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 21.06
[5] 소재지전체주소 : 충청북도 충주시 칠금동 440-1번지
[6] 도로명전체주소 : 충청북도 충주시 탄금대로 257 (칠금동  1층)
[7] 사업장명 : 채비원카페
[8] 최종수정시점 : 20160927132210
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.98552617720398
[11] 좌표정보(Y) : 127.9037422728433
[12] 시설총규모 : 21.06
[13] 홈페이지 : None
************************************************
71.4%
선택 index 0 : 칠금동 채비원카페 과의 일치율 71.4
https://m.place.naver.com/restaurant/38655045/menu/list
0 건강차 : 6,000원
1 커피 : 2,000원
save success
12486 번째.
[0] 관리번호 : 4390000-104-2016-00078
[1] 영업

78.6%
선택 index 0 : 수안보면 세븐일레븐충주수안보점 과의 일치율 78.6
12502 번째.
[0] 관리번호 : 4390000-104-2017-00011
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043847 3942
[4] 소재지면적 : 71.25
[5] 소재지전체주소 : 충청북도 충주시 성서동 306번지
[6] 도로명전체주소 : 충청북도 충주시 예성로 168  2층 29호 (성서동)
[7] 사업장명 : 중앙어울림시장상인회(플래닛나인)
[8] 최종수정시점 : 20200331093909
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.97185134744995
[11] 좌표정보(Y) : 127.93360405551181
[12] 시설총규모 : 71.25
[13] 홈페이지 : None
************************************************
일치결과 없음
12503 번째.
[0] 관리번호 : 4390000-104-2016-00070
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 80.25
[5] 소재지전체주소 : 충청북도 충주시 성내동 210번지
[6] 도로명전체주소 : 충청북도 충주시 중앙로 127 (성내동)
[7] 사업장명 : 커피와향기이야기
[8] 최종수정시점 : 20180822150724
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.971343214047266
[11] 좌표정보(Y) : 127.9372553538304
[12] 시설총규모 : 80.25
[13] 홈페이지 : None
************************************************
80.0%
선택 index 0 : 성내동 커피와향기이야기 과의 일치율 80.0
12504 번째.
[0] 관리번호 : 4390000-104-2017-00053
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 

...

50.0%
선택 index 0 : 청원구 미스바 과의 일치율 50.0
15935 번째.
[0] 관리번호 : 5725000-104-2013-02060
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 284 1880
[4] 소재지면적 : 71.60
[5] 소재지전체주소 : 충청북도 청주시 서원구 분평동 1375번지 (1층)
[6] 도로명전체주소 : 충청북도 청주시 서원구 1순환로1107번길 36  1층 (분평동)
[7] 사업장명 : 이디야커피청주분평점
[8] 최종수정시점 : 20180725111018
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.60786866861482
[11] 좌표정보(Y) : 127.4899296837825
[12] 시설총규모 : 71.6
[13] 홈페이지 : None
15936 번째.
[0] 관리번호 : 5725000-104-2013-02090
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 233 6296
[4] 소재지면적 : 257.90
[5] 소재지전체주소 : 충청북도 청주시 서원구 성화동 469번지 (1  2층)
[6] 도로명전체주소 : 충청북도 청주시 서원구 신성화로 39  1~2층 (성화동)
[7] 사업장명 : 할리스커피청주성화점
[8] 최종수정시점 : 20180725111114
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.61842150893959
[11] 좌표정보(Y) : 127.45803553387616
[12] 시설총규모 : 257.9
[13] 홈페이지 : None
15937 번째.
[0] 관리번호 : 5735000-104-2013-01080
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 212 2013
[4] 소재지면적 : 143.00
[5] 소재지전체주소 : 충청북도 청주시 청원구 율량동 1011번지 1층
[6] 도로명전체주소 : 충청북도 청주시 청원구 

일치결과 없음
15952 번째.
[0] 관리번호 : 5735000-104-2020-00005
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 80.00
[5] 소재지전체주소 : 충청북도 청주시 청원구 오창읍 양청리 810-2번지 오창 롯데캐슬 더 하이스트
[6] 도로명전체주소 : 충청북도 청주시 청원구 오창읍 중부로 760  201동 206호 (오창 롯데캐슬 더 하이스트)
[7] 사업장명 : ademain760 (아드망760)
[8] 최종수정시점 : 20200116152328
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.71006539249366
[11] 좌표정보(Y) : 127.43631853093163
[12] 시설총규모 : 80
[13] 홈페이지 : None
************************************************
일치결과 없음
15953 번째.
[0] 관리번호 : 5735000-104-2020-00013
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 218 7998
[4] 소재지면적 : 79.00
[5] 소재지전체주소 : 충청북도 청주시 청원구 내수읍 내수리 243-15번지
[6] 도로명전체주소 : 충청북도 청주시 청원구 내수읍 내수로 609-1
[7] 사업장명 : 이디야커피
[8] 최종수정시점 : 20200211140005
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.718681179620425
[11] 좌표정보(Y) : 127.5277445003305
[12] 시설총규모 : 79
[13] 홈페이지 : None
15954 번째.
[0] 관리번호 : 5725000-104-2020-00013
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 11.75
[5] 소재지전체주소 : 충청북도 청주시 서원구 사직동 748번지

70.0%
선택 index 0 : 청원구 팔공티 청주사천점 과의 일치율 70.0
https://m.place.naver.com/restaurant/1846472879/menu/list
0 루비에딸기퐁당라떼 : 3,800원
save success
15969 번째.
[0] 관리번호 : 5725000-104-2020-00005
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 23.43
[5] 소재지전체주소 : 충청북도 청주시 서원구 산남동 389번지 1층
[6] 도로명전체주소 : 충청북도 청주시 서원구 원흥로 72  1층 (산남동)
[7] 사업장명 : 카페 그레이스부엌
[8] 최종수정시점 : 20200131114841
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.612355555921276
[11] 좌표정보(Y) : 127.46930260191532
[12] 시설총규모 : 23.43
[13] 홈페이지 : None
************************************************
63.2%
선택 index 0 : 서원구 카페 그레이스부엌 과의 일치율 63.2
15970 번째.
[0] 관리번호 : 5720000-104-2020-00003
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 287 2115
[4] 소재지면적 : 96.17
[5] 소재지전체주소 : 충청북도 청주시 상당구 용암동 246번지 1층
[6] 도로명전체주소 : 충청북도 청주시 상당구 낙가산로 9  1층 (용암동)
[7] 사업장명 : 카페코지
[8] 최종수정시점 : 20200203170135
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.61870069795718
[11] 좌표정보(Y) : 127.5242034157146
[12] 시설총규모 : 96.17
[13] 홈페이지 : None
********************************

일치결과 없음
15984 번째.
[0] 관리번호 : 5725000-104-2020-00016
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 72.74
[5] 소재지전체주소 : 충청북도 청주시 서원구 수곡동 76-6번지 1층
[6] 도로명전체주소 : 충청북도 청주시 서원구 청남로 2038-1  1층 (수곡동)
[7] 사업장명 : 우주라이크커피(분평점)
[8] 최종수정시점 : 20200227205737
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 36.6140455611774
[11] 좌표정보(Y) : 127.48607479230343
[12] 시설총규모 : 72.74
[13] 홈페이지 : None
************************************************
74.1%
선택 index 0 : 서원구 우주라이크커피(분평점) 과의 일치율 74.1
https://m.place.naver.com/restaurant/1034160849/menu/list
0 토피넛라떼 : 4,300원
1 아인슈패너 : 4,800원
2 아메리캔 : 2,800원
3 더치커피 : 3,500원
4 더치라떼 : 4,300원
5 카푸치노 : 3,500원
6 카페라떼 : 3,500원
7 연유커피 : 4,300원
8 카페모카 : 4,300원
9 바닐라라떼 : 4,300원
10 헤이즐넛라떼 : 4,300원
11 카라멜마끼아또 : 4,300원
12 코코넛커피 : 4,500원
13 유자메리카노 : 4,500원
14 흑당카페라떼 : 5,500원
save success
15985 번째.
[0] 관리번호 : 5725000-104-2020-00017
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 043 262 3533
[4] 소재지면적 : 133.11
[5] 소재지전체주소 : 충청북도 청주시 서원구 사창동 462번지 1층
[6] 도로명전체주소 : 충청북도 청주시 서원구 1

60.0%
선택 index 0 : 서원구 왕커피 과의 일치율 60.0
https://m.map.naver.com/search2/site.nhn?query=%EC%84%9C%EC%9B%90%EA%B5%AC%20%EC%99%95%EC%BB%A4%ED%94%BC&sm=hty&style=v5&code=1654103338#/anchor/demenu/list
save success
